### Basic configuration

In [10]:
api_host = "https://staging.networkperspective.io"
network_id = "ad8c3e20-5651-4e48-a17d-44fafd0262da"

### Login

Get user creadentials

In [11]:
import getpass

# don't ask for username and password if they are already set
if 'username' not in locals():    
    username = input("Enter your username: ") 
    password = getpass.getpass('Enter your password: ')

Acquire access token

In [12]:
from np_client import np_client

configuration = np_client.Configuration(
    host = api_host
)

with np_client.ApiClient(configuration) as api_client:
    token_api = np_client.TokenApi(api_client)
    try:    
        token = token_api.token_get_token(
            username=username,  
            password=password, 
            grant_type="password", 
            client_id="ApiClient"
        )
    except Exception as e:
        print("Exception when calling TokenApi->token_get_token: %s\n" % e)  
        raise e

# set the token in the configuration to use it in the next calls
configuration.access_token = token.access_token

# token should be valid for 1 hour
print("Token expiration: ", token.expires)

Token expiration:  Wed, 13 Dec 2023 22:46:59 GMT


### Get available metrics and periods

In [13]:
with np_client.ApiClient(configuration) as api_client:    
    report_api = np_client.ReportApi(api_client)
    metrics_export_params_query = np_client.MetricsExportParamsQuery() 
    metrics_export_params_query.network_id = network_id
    metrics_export_params_query.aggregate = np_client.AggregateEnum.MONTH
    try:
        # Export metric params
        export_metric_params = report_api.report_export_metric_params(metrics_export_params_query=metrics_export_params_query)                
    except Exception as e:
        print("Exception when calling ReportApi->report_export_metric_params: %s\n" % e)
        raise e

print("periods", export_metric_params.periods)
print("metrics", export_metric_params.metrics)

periods ['2021-12', '2021-04', '2021-09', '2021-07', '2021-03', '2021-11', '2021-08', '2021-01', '2021-02', '2021-05', '2021-06', '2021-10']
metrics ['ManagerGuidanceMonthlyChats', 'WeeklyTeamMeetingSize', 'EmailsQueue', 'ManagerGuidanceDailyMeetings', 'MeetingDuration_below_30min', 'FocusedWorkStreakLength', 'ManagerGuidanceDailyOverall', 'MeetingsAfterHours', 'FocusedWorkDuration', 'MeetingDuration_over_4h', 'ManagerRoles', 'ModeOfWorkFocusedWork', 'ManagerGuidanceWeeklyOverall', 'MeetingDuration_2h_4h', 'ChannelsActiveBig', 'ManagerGuidanceDailyEmails', 'ModeOfWorkMultipleContextWork', 'ChatsDuration', 'DiversityOfConnections', 'Bottleneck', 'ModeOfWorkCollaboration', 'ManagerGuidanceWeeklyMeetings', 'TimeToPromotion', 'MeetingAttendees_9_18', 'ManagerGuidanceWeeklyEmails', 'Centrality', 'OneOnOneWeeklyDuration', 'FocusedWorkStreakCount', 'MaxNumberOfEmployees', 'MeetingsMultitasking', 'TimeOnMeetings', 'InternalTeamMeetings', 'ManagerGuidanceMonthlyOverall', 'ManagerGuidanceMonthly

In [14]:
last_period = export_metric_params.periods[-1]
all_metrics = export_metric_params.metrics

### Export direct metric values
These are metrics calculated for each team or employee group.

In [15]:
with np_client.ApiClient(configuration) as api_client:    
    report_api = np_client.ReportApi(api_client)

    metrics_export_query = np_client.MetricsExportQuery()     
    metrics_export_query.network_id = network_id
    metrics_export_query.periods = [last_period]
    metrics_export_query.metrics = all_metrics
    
    try:        
        metrics = report_api.report_export_metric_values(metrics_export_query=metrics_export_query)        
    except Exception as e:
        print("Exception when calling ReportApi->report_export_metric_values: %s\n" % e)
        raise e

Convert to dataframe for easy export & analysis

In [16]:
import pandas as pd

metrics_df = pd.DataFrame([m.to_dict() for m in metrics])
display(metrics_df.head())

,metric,period,value,groupId,groupName,category,clientGroupId
0,Bottleneck,2021-10,51.1287,0bf22a30-3da7-4125-a43c-0d0222b4a961,Account Executives (AEs) 1 (Business development),Team,None
1,Bottleneck,2021-10,51.3596,d63114a2-ba64-4253-9993-31171e110483,Account Executives (AEs) 2 (Business development),Team,None
2,Bottleneck,2021-10,20.2302,f7cad5c6-e7ca-46ac-a852-7b3419ff7d7e,Accounting (Finance),Team,None
3,Bottleneck,2021-10,20.3067,2d13b097-e6b6-4db1-95ec-2a5fed496b0e,Administration (Operations / back office),Team,None
4,Bottleneck,2021-10,58.9221,9b72dd3c-0270-4576-8059-d27f3c87b5b1,Agile coaches (People & org dev),Team,None


### Read realted metrics
These are cooperation paths between groups, read first 1000 values.

In [17]:
with np_client.ApiClient(configuration) as api_client:    
    report_api = np_client.ReportApi(api_client)

    metric_related_export_query = np_client.MetricRelatedExportQuery()     
    metric_related_export_query.network_id = network_id
    metric_related_export_query.metric = "CooperationPaths"
    metric_related_export_query.top = 1000
    
    try:        
        related_metrics = report_api.report_export_metric_related_values(metric_related_export_query=metric_related_export_query)        
    except Exception as e:
        print("Exception when calling ReportApi->report_export_metric_related_values: %s\n" % e)
        raise e

Convert to dataframe

In [18]:
related_df = pd.DataFrame([m.to_dict() for m in related_metrics])
display(related_df.head())

,period,value,groupId,groupName,relatedGroupId,relatedClientGroupId,relatedGroupName,category,clientGroupId
0,2021-10,483228.0,942240ca-9073-4df2-9bff-f9b346bf1432,ic,254cd9b4-37eb-4928-8214-ce87a4ec12bf,Persona,manager,Persona,None
1,2021-10,374502.0,254cd9b4-37eb-4928-8214-ce87a4ec12bf,manager,942240ca-9073-4df2-9bff-f9b346bf1432,Persona,ic,Persona,None
2,2021-10,362421.0,942240ca-9073-4df2-9bff-f9b346bf1432,ic,942240ca-9073-4df2-9bff-f9b346bf1432,Persona,ic,Persona,None
3,2021-10,353449.0,98ea8c92-3ae9-4130-aafa-0315064d8166,Project teams,98ea8c92-3ae9-4130-aafa-0315064d8166,Division,Project teams,Division,None
4,2021-10,323350.0,82b817d4-b2b2-47b2-8204-c4e214200365,People & org dev,82b817d4-b2b2-47b2-8204-c4e214200365,Division,People & org dev,Division,None
